# Optimizing preference matrix

The first step was to optimize weights, now we need to optimize preferences, so that the allocation is even greater (better).

## Problem definition

We have 14 producers and 14 consumers, we need to find the best possible preference dataframe (Prods X Cons) so that we minimize undistributed energy. The dataframe row has only 5 non-zero entries, which can have to be distinct numbers from set {1, 2, 3, 4, 5}.

```
example:
                Producers
                0	1	2	3	4	5	6	7
        0	0.0	5.0	3.0	0.0	2.0	1.0	4.0	0.0
        1	4.0	0.0	5.0	0.0	1.0	0.0	3.0	2.0
Cons    2	0.0	0.0	0.0	4.0	5.0	3.0	1.0	2.0
        3	1.0	0.0	4.0	0.0	0.0	2.0	3.0	5.0
        4	0.0	3.0	2.0	4.0	0.0	5.0	0.0	1.0
        5	4.0	2.0	3.0	5.0	0.0	0.0	0.0	1.0
        6	0.0	4.0	2.0	0.0	1.0	3.0	0.0	5.0
```

How good the matrix is will be determined in a process:

1. Input matrix
2. Calculate weights -> based on optimization algorithm
3. Calculate sum of unsatisfied demand and unallocated energy and that is the fitness



In [1]:
from weights_optimization_model import optimize_weights
import utils as ut
import preference_optimization as po
import time
import numpy as np

In [2]:
data_dir = '/home/miro/Bachelor/BT/data/outputs/'
excess_monthly, deficit_monthly = ut.load_excess_deficit(data_dir)

month = 4 # May
start = 0
end   = 1800 # Whole month non zero production or consumption

excess_df, deficit_df = ut.prepare_data(excess_monthly, deficit_monthly, month, start, end)
producers = excess_df.columns
consumers = deficit_df.columns

producers = list(excess_df.columns)
consumers = list(deficit_df.columns)

In [ ]:
preference_df = ut.generate_preferences(producers, consumers)
preference_df = preference_df.loc[producers,consumers]

opt_result = optimize_weights(excess_df, deficit_df, preference_df)
weights, unmet_demand, unused_supply = opt_result
fitness_score = ut.fitness(weights, preference_df, excess_df, deficit_df)

print(f'Objective function: {unmet_demand + unused_supply}')
print(f'fitness: {fitness_score}')

# BEST there could be np.float64(150.17971183115088)

Set parameter Username
Set parameter LicenseID to value 2652867
Academic license - for non-commercial use only - expires 2026-04-15
Objective function: 159.12753983462304
fitness: unsatisfied_demand     22.060502
available_energy      137.662956
Total                 159.723458
dtype: float64


In [ ]:
ga_params = {
    'generations': 20,
    'population_size': 30,
    'mutation_rate': 0.1,
    'crossover_rate': 0.8,
    'tournament_size': 5,
    'elitism_count': 2,
    'mutation_type': 'granular', 
    'initial_non_zero': 5
}

ga_result, ga_score, population = po.run_genetic_algorithm(
    excess_df, deficit_df, **ga_params
)

In [ ]:
opt_result = optimize_weights(excess_df, deficit_df, ga_result)
weights, unmet_demand, unused_supply = opt_result
fitness_score = ut.fitness(weights, ga_result, excess_df, deficit_df)
print(fitness_score)

# BEST there could be np.float64(150.17971183115088)
# OPTIMIZED: 152.417384

unsatisfied_demand     18.407465
available_energy      134.009920
Total                 152.417384
dtype: float64


In [4]:
# Test on small dataset

data_dir = '/home/miro/Bachelor/BT/data/outputs/' 
output_dir = '/home/miro/Bachelor/BT/Models/outputs/'

print(f"Loading data from: {data_dir}")
excess_monthly, deficit_monthly = ut.load_excess_deficit(data_dir)

month = 4
start_day = 0 
end_day   = 20


excess_df, deficit_df = ut.prepare_data(excess_monthly, deficit_monthly, month, start_day, end_day)
ideal_min_imbalance = np.abs(excess_df.sum(axis=1) - deficit_df.sum(axis=1)).sum()
print(f'Reference (Sum of Abs Imbalances per Time Step): {ideal_min_imbalance:.4f}')

Loading data from: /home/miro/Bachelor/BT/data/outputs/
Reference (Sum of Abs Imbalances per Time Step): 1.6439


In [5]:
print("\n" + "="*30)
print("Running Genetic Algorithm")
print("="*30)
ga_params = {
    'generations': 10,
    'population_size': 15,
    'mutation_rate': 0.1,
    'crossover_rate': 0.8,
    'tournament_size': 5,
    'elitism_count': 2,
    'mutation_type': 'granular', 
    'initial_non_zero': 5
}
print("GA Parameters:")
for k, v in ga_params.items():
    print(f"  {k}: {v}")

ga_start = time.time()
ga_result, ga_score, ga_final_population = po.run_genetic_algorithm(
    excess_df, deficit_df, **ga_params
)
ga_end = time.time()

final_ga_score = po.evaluate_matrix(ga_result, excess_df, deficit_df)
print(f'GA Score: {final_ga_score:.4f}')
print(f'GA Time: {(ga_end - ga_start) / 60:.4f} minutes')



Running Genetic Algorithm
GA Parameters:
  generations: 10
  population_size: 15
  mutation_rate: 0.1
  crossover_rate: 0.8
  tournament_size: 5
  elitism_count: 2
  mutation_type: granular
  initial_non_zero: 5
GA: Initializing population...
GA: Starting Generation 1/10
GA: New best fitness found: 1.6467
GA: Generation 1 finished in 24.76 seconds. Best fitness: 1.6467
GA: Starting Generation 2/10
GA: New best fitness found: 1.6456
GA: Generation 2 finished in 24.17 seconds. Best fitness: 1.6456
GA: Starting Generation 3/10
GA: Generation 3 finished in 27.46 seconds. Best fitness: 1.6456
GA: Starting Generation 4/10
GA: New best fitness found: 1.6454
GA: Generation 4 finished in 25.35 seconds. Best fitness: 1.6454
GA: Starting Generation 5/10
GA: Generation 5 finished in 23.48 seconds. Best fitness: 1.6454
GA: Starting Generation 6/10
GA: New best fitness found: 1.6454
GA: Generation 6 finished in 23.57 seconds. Best fitness: 1.6454
GA: Starting Generation 7/10
GA: New best fitness fo

In [6]:
# --- Run Simulated Annealing ---
print("\n" + "="*30)
print("Running Simulated Annealing")
print("="*30)
sa_params = {
    'iters': 350,          
    'init_temp': 100.0,
    'decay': 0.96,          
    'mutation_type': 'granular',
    'initial_non_zero': 5 
}

print("SA Parameters:")

for k, v in sa_params.items():
    print(f"  {k}: {v}")

sa_start = time.time()
sa_result, sa_score = po.simulated_annealing(
    excess_df, deficit_df, **sa_params
)
sa_end = time.time()

final_sa_score = po.evaluate_matrix(sa_result, excess_df, deficit_df)
print(f'SA Score: {final_sa_score:.4f}')
print(f'SA Time: {(sa_end - sa_start)/60:.4f} minutes')


Running Simulated Annealing
SA Parameters:
  iters: 350
  init_temp: 100.0
  decay: 0.96
  mutation_type: granular
  initial_non_zero: 5
SA: Initial fitness: 2.0187, Initial Temp: 100.0, Decay: 0.96
SA: Iter 11/350 - New best: 2.0181
SA: Iter 14/350 - New best: 2.0181
SA: Iter 26/350 - New best: 1.8835
SA: Iter 35/350 — Current best: 1.8835, Current fit: 1.8835, T=23.960350
SA: Iter 70/350 — Current best: 1.8835, Current fit: 2.1506, T=5.740984
SA: Iter 104/350 - New best: 1.6750
SA: Iter 105/350 — Current best: 1.6750, Current fit: 1.6750, T=1.375560
SA: Iter 109/350 - New best: 1.6545
SA: Iter 110/350 - New best: 1.6545
SA: Iter 119/350 - New best: 1.6518
SA: Iter 123/350 - New best: 1.6517
SA: Iter 128/350 - New best: 1.6510
SA: Iter 136/350 - New best: 1.6506
SA: Iter 140/350 — Current best: 1.6506, Current fit: 1.6506, T=0.329589
SA: Iter 143/350 - New best: 1.6506
SA: Iter 175/350 — Current best: 1.6506, Current fit: 1.6992, T=0.078971
SA: Iter 210/350 — Current best: 1.6506, Cu

In [7]:


# --- Run Hill Climbing ---
print("\n" + "="*30)
print("Running Local Search (Hill Climbing)")
print("="*30)
lc_params = {
    'iters': 350,
    'mutation_type': 'granular',
    'initial_non_zero': 5
}

print("HC Parameters:")
for k, v in lc_params.items():
    print(f"  {k}: {v}")

lc_start = time.time()
lc_result, lc_score = po.hill_climb(
    excess_df, deficit_df, **lc_params
)
lc_end = time.time()

final_lc_score = po.evaluate_matrix(lc_result, excess_df, deficit_df)
print(f'LC Score: {final_lc_score:.4f}')
print(f'LC Time: {(lc_end - lc_start) / 60 :.4f} minutes')


Running Local Search (Hill Climbing)
HC Parameters:
  iters: 350
  mutation_type: granular
  initial_non_zero: 5
HC: Initial fitness: 1.6885
HC: Iter 35/350 — Current best: 1.6848
HC: Iter 70/350 — Current best: 1.6822
HC: Iter 105/350 — Current best: 1.6484
HC: Iter 140/350 — Current best: 1.6471
HC: Iter 175/350 — Current best: 1.6470
HC: Iter 210/350 — Current best: 1.6470
HC: Iter 245/350 — Current best: 1.6467
HC: Iter 280/350 — Current best: 1.6466
HC: Iter 315/350 — Current best: 1.6461
HC: Iter 350/350 — Current best: 1.6461
HC: Final best fitness: 1.6461
LC Score: 1.6461
LC Time: 9.0594 minutes


In [ ]:
ga_weights, _, _ = optimize_weights(excess_df, deficit_df, ga_result)
sa_weights, _, _ = optimize_weights(excess_df, deficit_df, sa_result)
lc_weights, _, _ = optimize_weights(excess_df, deficit_df, lc_result)